Step0-- analyze and store info about the camera pixel-by-pixel DC model, saturation point)
Step1-- import raw data from h5 files, clip the saturated pixels, subtract the DC noise, save data as additional npy arrays
Step2-- *** Use the data to generate camera response functions and convert the multiple exposure times into a radiance map.**

In [1]:

import os
import numpy as np
import logging

 # process_and_save.py

from Step2_image_radiance_dev import load_clipped_denoised_data, process_hdr_images, save_crf_data

# Your existing setup code here
directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/'
experiment_title = 'Water_immersed'
base_data_folder = '240817_data_95xSmax'

# Add a parameter for the number of image sets to process
num_sets_to_process = 1  # Change this value to process more or fewer sets

# Load the processed data
loaded_data = load_clipped_denoised_data(directory, experiment_title, base_data_folder)

# Process HDR images and get all necessary data
processed_data = process_hdr_images(directory, experiment_title, base_data_folder, smoothing_lambda=1000, num_sets=num_sets_to_process)

# Save CRF data for future analysis
crf_file = save_crf_data(processed_data, directory, experiment_title, base_data_folder)

print(f"CRF data saved to: {crf_file}")


Processing Water_immersed_808_BP1300
Images shape: (15, 640, 512)
Exposure times shape: (15,)
Slinear shape: (640, 512)
Sd shape: (640, 512)
bias shape: (640, 512)
Zmax_precomputed shape: (15, 640, 512)
Number of processed data items: 1

Processing data item 1:
  key: Water_immersed_808_BP1300
  radiance_map shape: (640, 512)
  response_curve shape: (4652,)
  z_min: 0
  z_max: 4651.78439803009
  intensity_samples shape: (20221, 15)
  log_exposures shape: (20221, 15)

Saving CRF data...
CRF data saved to: /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/240817_data_95xSmax/final_data/Water_immersed_crf_data.npz
Verified saved data. Keys in the file:
  key_0
  radiance_map_0
  response_curve_0
  z_min_0
  z_max_0
  intensity_samples_0
  log_exposures_0
CRF data saved to: /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image

In [2]:
# generate_report.py

import os
import numpy as np
import logging
from Step2_image_radiance_dev import load_crf_data, generate_multi_page_report, adaptive_weight

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Your existing setup code here
directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/'
experiment_title = 'Water_immersed'
base_data_folder = '240817_data_95xSmax'

# Load the saved CRF data
crf_file = os.path.join(directory, base_data_folder, "final_data", f"{experiment_title}_crf_data.npz")
processed_data = load_crf_data(crf_file)

logger.info("Loaded processed data:")
for i, data in enumerate(processed_data):
    logger.info(f"\nData item {i+1}:")
    for key, value in data.items():
        if isinstance(value, np.ndarray):
            logger.info(f"  {key} shape: {value.shape}")
            if np.issubdtype(value.dtype, np.number):  # Check if the array contains numeric data
                if np.isnan(value).any() or np.isinf(value).any():
                    logger.warning(f"  Warning: {key} contains NaN or inf values")
                logger.info(f"  {key} min: {np.min(value)}, max: {np.max(value)}")
            else:
                logger.info(f"  {key} contains non-numeric data")
        else:
            logger.info(f"  {key}: {value}")

# Generate the multi-page report
if processed_data:
    try:
        generate_multi_page_report(processed_data, directory, experiment_title, adaptive_weight, base_data_folder)
        logger.info("Report generation complete")
    except Exception as e:
        logger.error(f"Error generating report: {str(e)}")
        import traceback
        logger.error(traceback.format_exc())
else:
    logger.warning("No data to generate report. Please check the CRF data file.")

logger.info("Script execution complete")

INFO:__main__:Loaded processed data:
INFO:__main__:
Data item 1:
INFO:__main__:  key shape: ()
INFO:__main__:  key contains non-numeric data
INFO:__main__:  radiance_map shape: (640, 512)
INFO:__main__:  radiance_map min: 3.256209373474121, max: 11.000643730163574
INFO:__main__:  response_curve shape: (4652,)
INFO:__main__:  response_curve min: 2.853682872470154, max: 9.247411430647702
INFO:__main__:  z_min shape: ()
INFO:__main__:  z_min min: 0, max: 0
INFO:__main__:  z_max shape: ()
INFO:__main__:  z_max min: 4651.78439803009, max: 4651.78439803009
INFO:__main__:  intensity_samples shape: (20221, 15)
INFO:__main__:  intensity_samples min: 0.0, max: 3848.6015625
INFO:__main__:  log_exposures shape: (20221, 15)
INFO:__main__:  log_exposures min: -5.00555533281701, max: 11.656276335736774


Loading CRF data from: /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/240817_data_95xSmax/final_data/Water_immersed_crf_data.npz
Keys in the loaded file:
  key_0
  radiance_map_0
  response_curve_0
  z_min_0
  z_max_0
  intensity_samples_0
  log_exposures_0
Processing data item 0
  Loaded key_0: ndarray with shape (), dtype: <U25
  Loaded radiance_map_0: ndarray with shape (640, 512), dtype: float32
  Loaded response_curve_0: ndarray with shape (4652,), dtype: float64
  Loaded z_min_0: ndarray with shape (), dtype: int64
  Loaded z_max_0: ndarray with shape (), dtype: float64
  Loaded intensity_samples_0: ndarray with shape (20221, 15), dtype: float32
  Loaded log_exposures_0: ndarray with shape (20221, 15), dtype: float64
Loaded 1 processed data items
Debug: intensity_samples shape: (20221, 15)
Debug: log_exposures shape: (20221, 15)
Debug: response_curve shape: (4652,)
Deb

INFO:Step2_image_radiance_dev:Added report page for Water_immersed_808_BP1300
INFO:Step2_image_radiance_dev:Finished processing all files.
INFO:Step2_image_radiance_dev:Report saved as /Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed/240817_data_95xSmax/reports/hdr_report_Water_immersed_20240817_193923.pdf
INFO:__main__:Report generation complete
INFO:__main__:Script execution complete
